**Финальный результат:** test accuracy 0.994 <br>
Чем больше берём слов в max_words, тем лучше результат.

max_len достаточно и 10. Эпох тоже

Dropout в модели не сильно влияет. Возможно влияет кол-во слоев

Подозреваю, что не сложный датасет, поэтому результаты хорошие, даже простая самописная нейронка справляется

In [186]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch.nn.functional as F

In [187]:
max_words = 4000
max_len = 10
num_classes = 1

# Training
epochs = 10
batch_size = 512
print_batch_n = 100

th = 0.5

In [188]:
import pandas as pd

df_train = pd.read_csv("train.csv")
final_test = pd.read_csv("test.csv")


In [189]:
X_train, X_test, y_train, y_test = train_test_split(df_train['tweet'], df_train['label'], test_size=0.25, random_state=26)

In [190]:
X_train.head()

10549    @user so sweet. .  fathers day ..love u devð...
24767                              porn vids web free sex 
13400    @user chicken breast with homemade bbq sauce, ...
25055    i am thankful for comfo. #thankful #positive     
15595    quick scan board for empathy now up on pintere...
Name: tweet, dtype: object

In [191]:
X_test.head()

22757    visiting a friend and finding your book in the...
7670     can't help #smiling when i #color this!   #sat...
16618    saving a #year with a single #drive  #fun #fri...
11168    #winterfashion   bull up: you will dominate yo...
11138                    gonna be doing new content soon  
Name: tweet, dtype: object

In [192]:
!pip install stop_words pymorphy2

In [193]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [194]:
sw = set(get_stop_words("en"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\snot", "not", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

In [195]:
X_train.head(2)

10549    user sweet fathers day love u devðððð...
24767                               porn vids web free sex
Name: tweet, dtype: object

In [196]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [197]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [198]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [199]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [200]:
tokens_filtered_top[:10]

['user', 'love', 'day', 'happy', 'amp', 'just', 'will', 'u', 'time', 'im']

In [201]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [202]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [203]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [204]:
x_train[12345]

array([   0,    0, 3678, 3965,  338,   25,  122,  750, 2158, 1954],
      dtype=int32)

In [205]:
x_train.shape

(23971, 10)

In [206]:
import random
import torch
import torch.nn as nn

seed = 26

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [207]:
class Net(nn.Module):
    def __init__(self, vocab_size=20, embedding_dim = 128, out_channel = 128, num_classes = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dp = nn.Dropout(0.25)
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
        self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear1 = nn.Linear(out_channel, out_channel//2)
        self.linear2 = nn.Linear(out_channel//2, num_classes)

    def forward(self, x):
        output = self.embedding(x)
        #                       B  F  L
        output = output.permute(0, 2, 1)
        output = self.dp(output)
        output = self.conv_1(output)
        output = self.relu(output)
        output = self.pool(output)
        output = self.conv_2(output)
        output = self.relu(output)
        output = self.pool(output)
        output = torch.max(output, axis=2).values
        output = self.linear1(output)
        output = self.relu(output)
        output = self.linear2(output)
        output = F.sigmoid(output)

        return output

In [208]:
from torch.utils.data import DataLoader, Dataset

class DataWrapper(Dataset):
    def __init__(self, data, target=None, transform=None):
        self.data = torch.from_numpy(data).long()
        if target is not None:
            self.target = torch.from_numpy(target).long()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index] if self.target is not None else None

        if self.transform:
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [209]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cuda'

In [210]:
model = Net(vocab_size=max_words).to(device)

print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

model.train()
#model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=10e-3)
criterion = nn.BCELoss()


train_dataset = DataWrapper(x_train, y_train.values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = DataWrapper(x_test, y_test.values)
test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

Net(
  (embedding): Embedding(4000, 128)
  (dp): Dropout(p=0.25, inplace=False)
  (conv_1): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (linear1): Linear(in_features=128, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=1, bias=True)
)
Parameters: 602497


In [211]:

manual_acc = []
for epoch in tqdm(range(epochs)):
    model.train()
    right, num = 0, 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels.float().view(-1,1))
        loss.backward()
        optimizer.step()
        right += (labels == torch.squeeze((outputs > th).int())).sum()
        num += len(labels)

    model.eval()
    loss_accumed, test_right, test_num = 0, 0, 0
    for i, data in enumerate(test_loader, 0):
        X, y = data[0].to(device), data[1].to(device)
        output = model(X)
        loss = criterion(output, y.float().view(-1,1))
        loss_accumed += loss
        test_num += len(y)
        test_right += (y == torch.squeeze((output > th).int())).sum()

    train_acc = right/num
    test_acc = test_right/test_num
    manual_acc.append(test_acc)
    print(f"Epoch {epoch+1}; test_loss {loss_accumed:.3f}; test_acc {test_acc:.3f}; train_acc {train_acc:.3f}")

print('Training is finished!')

 10%|█         | 1/10 [00:00<00:08,  1.05it/s]

Epoch 1; test_loss 8.173; test_acc 0.929; train_acc 0.911


 20%|██        | 2/10 [00:02<00:11,  1.39s/it]

Epoch 2; test_loss 5.966; test_acc 0.951; train_acc 0.933


 30%|███       | 3/10 [00:03<00:08,  1.23s/it]

...

Epoch 8; test_loss 1.235; test_acc 0.992; train_acc 0.984


 90%|█████████ | 9/10 [00:10<00:01,  1.16s/it]

Epoch 9; test_loss 1.264; test_acc 0.994; train_acc 0.987


100%|██████████| 10/10 [00:11<00:00,  1.12s/it]

Epoch 10; test_loss 0.956; test_acc 0.994; train_acc 0.988
Training is finished!
